In [1]:
# ---------- CELL 1: Install & Preflight ----------
# Run this first. It installs cloudflared and Python libs.
# It may take a few minutes (model downloads happen in later cell).

# System packages + cloudflared
!apt-get update -qq
!apt-get install -y -qq wget ca-certificates ffmpeg fonts-noto

# Download and install cloudflared (auto tunnel)
!wget -q -O /tmp/cloudflared.deb "https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb"
!dpkg -i /tmp/cloudflared.deb || true
!apt-get -f install -y -qq

# Python packages (transformers may be large)
!pip install -q yt-dlp moviepy==1.0.3 ffmpeg-python transformers[torch] accelerate sentencepiece soundfile gTTS flask

# Try installing Bark (optional, better TTS). If it fails, code will fallback to gTTS.
try:
    !pip install -q git+https://github.com/suno-ai/bark.git
except Exception as e:
    print("Bark install attempt failed (no problem - gTTS fallback will be used).", e)

# Quick checks
import sys, torch
print("Python:", sys.version.split()[0])
print("Torch:", getattr(torch, "__version__", "not installed"))
print("GPU available:", torch.cuda.is_available())

# Create output folder
import os
os.makedirs('/content/auto_videos', exist_ok=True)
print("Output directory:", "/content/auto_videos")


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
(Reading database ... 127572 files and directories currently installed.)
Preparing to unpack /tmp/cloudflared.deb ...
Unpacking cloudflared (2025.10.1) over (2025.10.1) ...
Setting up cloudflared (2025.10.1) ...
Processing triggers for man-db (2.10.2-1) ...
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Python: 3.12.12
Torch: 2.8.0+cu126
GPU available: True
Output directory: /content/auto_videos


In [2]:
# ---------- CELL 2 (REPLACE previous): Production Queue Server (TEXT-ONLY) + cloudflared ----------
# Paste & run this in Notebook A (replace old CELL 2). This version returns text facts only.

import os, time, uuid, json, subprocess, threading, queue, re
from flask import Flask, request, jsonify

# ---------- CONFIG ----------
OUTPUT_DIR = "/content/auto_videos"
os.makedirs(OUTPUT_DIR, exist_ok=True)
PORT = 5000

# Simple in-memory queue and jobs store
job_q = queue.Queue()
jobs = {}

# ---------- Simple text fact generator (replaceable) ----------
def generate_fact(topic):
    # Temporary deterministic facts — later we'll swap with Groq-based function
    facts = [
        f"Overthinking makes your brain treat mere thoughts like real threats.",
        f"People who overthink often get decision paralysis from fearing the worst outcome.",
        f"Overthinking commonly disrupts sleep and drains emotional energy."
    ]
    # return a list of strings
    return facts

# ---------- Worker (TEXT ONLY) ----------
def worker_text_only():
    while True:
        jobid, topic = job_q.get()
        try:
            jobs[jobid]['status'] = 'running'
            print(f"🔧 Worker processing job {jobid} — topic: {topic}")

            # Generate text-only output
            result = generate_fact(topic)
            # Save result & mark done
            jobs[jobid]['result'] = result
            jobs[jobid]['status'] = 'done'
            jobs[jobid]['finished'] = time.time()

            print(f"✅ Job {jobid} done — result ready.")
        except Exception as e:
            jobs[jobid]['status'] = 'error'
            jobs[jobid]['error'] = str(e)
            print(f"❌ Error processing job {jobid}: {e}")
        finally:
            job_q.task_done()

# Start the single worker thread (daemon)
threading.Thread(target=worker_text_only, daemon=True).start()

# ---------- Flask app: enqueue + status endpoints ----------
app = Flask(__name__)

@app.route("/generate", methods=["POST"])
def enqueue():
    data = request.get_json(force=True, silent=True) or {}
    topic = data.get('topic','creepy facts')
    jobid = str(uuid.uuid4())[:8]
    jobs[jobid] = {
        "status":"queued",
        "topic":topic,
        "created": time.time()
    }
    job_q.put((jobid, topic))
    return jsonify({"status":"queued","jobid":jobid,"poll":f"/status/{jobid}"})

@app.route("/status/<jobid>", methods=["GET"])
def status(jobid):
    info = jobs.get(jobid)
    if not info:
        return jsonify({"error":"unknown jobid"}), 404
    return jsonify(info)

@app.route("/", methods=["GET"])
def root():
    return jsonify({"status":"alive","info":"Production queue API. POST /generate with {'topic':'...'}"})

# ---------- Start cloudflared tunnel (prints public URL) ----------
def start_cloudflared_and_print():
    cmd = ["cloudflared", "tunnel", "--url", f"http://localhost:{PORT}"]
    popen = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    public_url = None
    for _ in range(400):
        line = popen.stdout.readline()
        if not line:
            time.sleep(0.1)
            continue
        print(line.strip())
        if "trycloudflare.com" in line or "trycloudflare" in line:
            m = re.search(r'(https?://[^\s]+trycloudflare[^\s]*)', line)
            public_url = m.group(1) if m else line.strip()
            print("\n✅ PUBLIC URL:", public_url)
            print("Use this to POST /generate and GET /status/<jobid>\n")
            break

# Run Flask app and cloudflared in threads
def start_flask():
    app.run(host='0.0.0.0', port=PORT)

threading.Thread(target=start_flask, daemon=True).start()
time.sleep(1)
threading.Thread(target=start_cloudflared_and_print, daemon=True).start()

print("✅ TEXT-ONLY production queue server started. POST /generate to queue jobs.")


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


✅ TEXT-ONLY production queue server started. POST /generate to queue jobs.


In [3]:
# ---------- CELL: Upgrade worker -> Full Video w/ Bark TTS + Ambient Music (Dark ambient A) ----------
# Paste & run in Notebook A (replaces previous video worker). Keep the notebook open.

import os, time, uuid, json, subprocess, threading, queue, re
from flask import Flask, request, jsonify, send_file
from moviepy.editor import VideoFileClip, AudioFileClip, CompositeVideoClip, concatenate_videoclips, ColorClip, ImageClip, CompositeAudioClip, afx
from PIL import Image, ImageDraw, ImageFont
import soundfile as sf
import numpy as np

# CONFIG
OUTPUT_DIR = "/content/auto_videos"
AUDIO_DIR = os.path.join(OUTPUT_DIR, "audio")
os.makedirs(AUDIO_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)
PORT = 5000
WIDTH, HEIGHT, FPS = 1080, 1920, 25
FACTS_COUNT = 5
USERNAME_WM = os.getenv("USERNAME_WM", "darktruths.hub007")
BARK_VOICE = "v2/en_speaker_9"   # deep male documentary
MUSIC_SEARCH_QUERY = "dark ambient background music creative commons"
MUSIC_MAX_SECONDS = 200

# Ensure Bark available flag
BARK_AVAILABLE = False
try:
    from bark import generate_audio, preload_models, SAMPLE_RATE
    try:
        print("Preloading Bark models (may download tens to hundreds MB).")
        preload_models()
    except Exception as e:
        print("Bark preload warning:", e)
    BARK_AVAILABLE = True
    print("Bark is available and will be used for TTS.")
except Exception as e:
    print("Bark not available, will try to fall back to gTTS if necessary.", e)
    BARK_AVAILABLE = False

# Safe generate_facts: use existing function if present, else fallback simple generator
def safe_generate_facts(topic, n=FACTS_COUNT):
    try:
        # if a user-defined generate_facts exists, call it
        gf = globals().get("generate_facts", None)
        if callable(gf):
            out = gf(topic, n=n)
            if isinstance(out, list) and len(out) >= 1:
                return out[:n]
    except Exception as e:
        print("Existing generate_facts() raised:", e)
    # fallback simple facts
    facts = []
    for i in range(n):
        facts.append(f"Fact {i+1} about {topic}.")
    return facts

# ---- Bark TTS helper (write wav) ----
def synthesize_bark_to_wav(text, out_wav_path, voice=BARK_VOICE):
    try:
        # generate_audio returns numpy array or list of floats
        wav = generate_audio(text=text, history_prompt=voice)
        # ensure numpy array
        arr = np.array(wav)
        sf.write(out_wav_path, arr, SAMPLE_RATE)
        return out_wav_path
    except Exception as e:
        print("Bark synth error:", e)
        return None

# convert wav to mp3 (ffmpeg must be installed in Colab)
def wav_to_mp3(wav_path, mp3_path):
    try:
        cmd = ["ffmpeg", "-y", "-i", wav_path, "-codec:a", "libmp3lame", "-qscale:a", "2", mp3_path]
        subprocess.run(cmd, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        return mp3_path
    except Exception as e:
        print("ffmpeg convert error:", e)
        return None

# ---- Download a single ambient music track (yt-dlp search for CC) ----
def download_ambient_music(query=MUSIC_SEARCH_QUERY):
    try:
        out_template = os.path.join(OUTPUT_DIR, "ambient_%(id)s.%(ext)s")
        cmd = [
            "yt-dlp",
            f"ytsearch1:{query}",
            "--no-playlist",
            "--restrict-filenames",
            "--format", "bestaudio",
            "--output", out_template
        ]
        subprocess.run(cmd, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        # find downloaded file
        candidates = sorted([os.path.join(OUTPUT_DIR,f) for f in os.listdir(OUTPUT_DIR) if f.startswith("ambient_")])
        if not candidates:
            return None
        music_file = candidates[-1]
        # trim large file to MUSIC_MAX_SECONDS seconds to speed up
        trimmed = music_file.replace(".webm", "_trim.mp3").replace(".m4a", "_trim.mp3").replace(".mp3","_trim.mp3")
        # use ffmpeg to trim
        cmd2 = ["ffmpeg", "-y", "-i", music_file, "-t", str(MUSIC_MAX_SECONDS), "-vn", "-acodec", "libmp3lame", "-qscale:a", "4", trimmed]
        subprocess.run(cmd2, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        return trimmed if os.path.exists(trimmed) else music_file
    except Exception as e:
        print("Ambient music download failed:", e)
        return None

# ---- Create text image using PIL (avoids ImageMagick) ----
def render_text_image(text, out_image_path, width=WIDTH, padding=80, font_size=64):
    # choose font (DejaVuSans included in Colab)
    try:
        font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf", font_size)
    except:
        font = ImageFont.load_default()
    # wrap text manually
    img = Image.new("RGBA", (width, 600), (0,0,0,0))
    draw = ImageDraw.Draw(img)
    # split into lines
    words = text.split()
    lines = []
    cur = ""
    for w in words:
        test = (cur + " " + w).strip()
        wsize = draw.textsize(test, font=font)[0]
        if wsize > (width - 2*padding):
            lines.append(cur)
            cur = w
        else:
            cur = test
    if cur:
        lines.append(cur)
    # compute height
bbox = draw.textbbox((0, 0), "Ay", font=font)
line_h = (bbox[3] - bbox[1]) + 10   # height + padding

img_h = padding + line_h * len(lines) + padding
img = Image.new("RGBA", (width, max(img_h, 240)), (0,0,0,0))
draw = ImageDraw.Draw(img)

y = padding // 2
for line in lines:
    bbox = draw.textbbox((0, 0), line, font=font)
    w = bbox[2] - bbox[0]
    h = bbox[3] - bbox[1]
    draw.text(((width - w) // 2, y), line, font=font, fill=(255,255,255,255))
    y += line_h

img.save(out_image_path)
return out_image_path

# ---- Build final vertical clip, per-fact audio sync ----
def build_vertical_video_v2(facts, voice_mp3_files, broll_clips, out_path, username=USERNAME_WM):
    clips = []
    audio_clips = []
    music_clip = None
    # load background music if present
    music_path = download_ambient_music()
    if music_path:
        try:
            music_clip = AudioFileClip(music_path).subclip(0, 9999)
            music_clip = music_clip.fx(afx.audio_fadein, 0.5).volumex(0.15)
        except Exception as e:
            print("music load err:", e)
            music_clip = None

    for idx, fact in enumerate(facts):
        # choose b-roll clip or fallback color
        bg = None
        if broll_clips and len(broll_clips) > idx:
            try:
                vc = VideoFileClip(broll_clips[idx]).resize(height=HEIGHT)
                if vc.w < WIDTH:
                    vc = vc.resize(width=WIDTH)
                clip_dur = AudioFileClip(voice_mp3_files[idx]).duration
                vc = vc.subclip(0, min(clip_dur + 0.5, vc.duration)).set_duration(clip_dur + 0.5)
                bg = vc
            except Exception as e:
                print("bg clip load err:", e)
        if bg is None:
            bg = ColorClip((WIDTH, HEIGHT), color=(10,10,12)).set_duration(AudioFileClip(voice_mp3_files[idx]).duration + 0.5)

        # render text image and make ImageClip
        imgfile = os.path.join(OUTPUT_DIR, f"text_{uuid.uuid4().hex[:6]}_{idx}.png")
        render_text_image(fact, imgfile, width=WIDTH, font_size=56)
        txt_clip = ImageClip(imgfile).set_duration(AudioFileClip(voice_mp3_files[idx]).duration + 0.5).set_position(("center", int(HEIGHT*0.12)))

        # voice audio
        aclip = AudioFileClip(voice_mp3_files[idx])
        # mix music (if available) with voice: voice louder, music low
        if music_clip:
            # create a music subclip matching aclip.duration and reduce volume
            try:
                music_sub = music_clip.subclip(0, aclip.duration).volumex(0.12)
                combined_audio = CompositeAudioClip([music_sub, aclip])
            except Exception as e:
                print("music combine err:", e)
                combined_audio = aclip
        else:
            combined_audio = aclip

        clip = CompositeVideoClip([bg, txt_clip], size=(WIDTH, HEIGHT)).set_duration(aclips := aclip.duration + 0.5)
        clip = clip.set_audio(combined_audio)
        clips.append(clip)

    # concatenate and write file
    if not clips:
        raise RuntimeError("No clips generated")
    final = concatenate_videoclips(clips, method="compose")
    # add watermark as final overlay
    try:
        wm = ImageClip(render_text_image(username, os.path.join(OUTPUT_DIR,"wm.png"), width=600, font_size=28)).set_duration(final.duration).set_position(("right", "bottom")).set_opacity(0.85)
        final = CompositeVideoClip([final, wm])
    except Exception:
        pass

    final.write_videofile(out_path, fps=FPS, codec="libx264", audio_codec="aac", threads=4)
    # cleanup small temp files maybe
    return out_path

# ---- Helper: download small set of b-roll clips (1 per fact) using yt-dlp (may return fewer) ----
def download_brolls_for_topic(topic, count=FACTS_COUNT, max_seconds=8):
    clips = []
    for q in [f"{topic} cinematic", "dark cinematic fog", "abandoned building cinematic"]:
        try:
            out_template = os.path.join(OUTPUT_DIR, "ytclip_%(id)s.%(ext)s")
            cmd = [
                "yt-dlp",
                f"ytsearch1:{q} creative commons",
                "--no-playlist",
                "--restrict-filenames",
                "--format", "mp4",
                "--output", out_template
            ]
            subprocess.run(cmd, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
            # find latest matching file
            candidates = sorted([os.path.join(OUTPUT_DIR,f) for f in os.listdir(OUTPUT_DIR) if f.startswith("ytclip_") and f.endswith(".mp4")])
            if candidates:
                fpath = candidates[-1]
                # trim
                trimmed = fpath.replace(".mp4", "_trim.mp4")
                clip = VideoFileClip(fpath)
                dur = min(max_seconds, clip.duration)
                clip.subclip(0, dur).write_videofile(trimmed, fps=FPS, codec="libx264", audio_codec="aac", threads=2, verbose=False, logger=None)
                clip.close()
                clips.append(trimmed)
            if len(clips) >= count:
                break
        except Exception as e:
            print("broll fetch error:", e)
            continue
    return clips

# ---- Worker: full pipeline (facts -> per-fact bark mp3 -> b-roll -> final video) ----
job_q = queue.Queue()
jobs = {}  # jobid -> info

def full_worker():
    while True:
        jobid, topic = job_q.get()
        jobs[jobid]['status'] = 'running'
        try:
            print(f"🔧 Processing job {jobid} topic:{topic}")
            # 1) generate facts (use safe wrapper)
            facts = safe_generate_facts(topic, n=FACTS_COUNT)

            # 2) per-fact TTS (Bark -> wav -> mp3)
            voice_files = []
            for i, fact in enumerate(facts):
                wav_path = os.path.join(AUDIO_DIR, f"{jobid}_fact{i+1}.wav")
                mp3_path = os.path.join(AUDIO_DIR, f"{jobid}_fact{i+1}.mp3")
                ok = None
                if BARK_AVAILABLE:
                    ok = synthesize_bark_to_wav(fact, wav_path, voice=BARK_VOICE)
                if not ok:
                    # fallback to simple gTTS
                    from gtts import gTTS
                    t = gTTS(text=fact, lang="en")
                    t.save(wav_path)
                # convert to mp3
                wav_to_mp3(wav_path, mp3_path)
                voice_files.append(mp3_path)

            # 3) download b-roll clips (may be fewer than facts)
            brolls = download_brolls_for_topic(topic, count=FACTS_COUNT, max_seconds=8)

            # 4) build final video synchronized to per-fact audio
            out_file = os.path.join(OUTPUT_DIR, f"creepy_{jobid}.mp4")
            build_vertical_video_v2(facts, voice_files, brolls, out_file)

            jobs[jobid]['status'] = 'done'
            jobs[jobid]['result'] = out_file
            jobs[jobid]['finished'] = time.time()
            print(f"✅ Job {jobid} finished -> {out_file}")
        except Exception as e:
            jobs[jobid]['status'] = 'error'
            jobs[jobid]['error'] = str(e)
            print("❌ Worker error:", e)
        finally:
            job_q.task_done()

# start worker thread (daemon)
threading.Thread(target=full_worker, daemon=True).start()

# ---- Flask endpoints (enqueue + status + download) ----
app = Flask(__name__)

@app.route("/generate", methods=["POST"])
def enqueue():
    data = request.get_json(force=True, silent=True) or {}
    topic = data.get('topic','creepy facts')
    jobid = str(uuid.uuid4())[:8]
    jobs[jobid] = {"status":"queued","topic":topic,"created":time.time()}
    job_q.put((jobid, topic))
    return jsonify({"status":"queued","jobid":jobid,"poll":f"/status/{jobid}"})

@app.route("/status/<jobid>", methods=["GET"])
def status(jobid):
    info = jobs.get(jobid)
    if not info:
        return jsonify({"error":"unknown jobid"}), 404
    return jsonify(info)

@app.route("/download/<jobid>", methods=["GET"])
def download(jobid):
    info = jobs.get(jobid)
    if not info or info.get("status") != "done":
        return jsonify({"error":"not ready"}), 404
    path = info.get("result")
    if not path or not os.path.exists(path):
        return jsonify({"error":"file missing"}), 404
    return send_file(path, mimetype="video/mp4", as_attachment=True, download_name=os.path.basename(path))

@app.route("/", methods=["GET"])
def root():
    return jsonify({"status":"alive","info":"Full video queue API. POST /generate with {'topic':'...'}"})

# Start cloudflared tunnel printing (if not already started elsewhere)
def start_cloudflared_and_print_local():
    try:
        cmd = ["cloudflared", "tunnel", "--url", f"http://localhost:{PORT}"]
        popen = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
        for _ in range(400):
            line = popen.stdout.readline()
            if not line:
                time.sleep(0.1)
                continue
            print(line.strip())
            if "trycloudflare.com" in line or "trycloudflare" in line:
                m = re.search(r'(https?://[^\s]+trycloudflare[^\s]*)', line)
                public_url = m.group(1) if m else line.strip()
                print("\n✅ PUBLIC URL:", public_url)
                print("Use this to POST /generate and GET /status/<jobid>\n")
                break
    except Exception as e:
        print("cloudflared start failed (maybe already running):", e)

# If the Flask server isn't running, start it. If it is, you can ignore.
def start_flask_local():
    app.run(host='0.0.0.0', port=PORT)

# run flask & cloudflared if needed (safe to call)
threading.Thread(target=start_flask_local, daemon=True).start()
time.sleep(1)
threading.Thread(target=start_cloudflared_and_print_local, daemon=True).start()

print("✅ Full-video worker + endpoints started. POST /generate to queue a job.")

2025-11-01T17:18:48Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-11-01T17:18:48Z INF Requesting new quick Tunnel on trycloudflare.com...

✅ PUBLIC URL: 2025-11-01T17:18:48Z INF Requesting new quick Tunnel on trycloudflare.com...
Use this to POST /generate and GET /status/<jobid>

Preloading Bark models (may download tens to hundreds MB).
Bark preload warning: Weights only load failed. This file can still be loaded, to do so you have two options, do those

NameError: name 'draw' is not defined

In [ ]:
# =========================
# 🌐 CLOUDFLARED TUNNEL SETUP
# =========================
import subprocess, time, re

# Install cloudflared if not already installed
!curl -L https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -o cloudflared
!chmod +x cloudflared

PORT = 5000  # same port as Flask API

def start_tunnel():
    print("Starting Cloudflared tunnel...")
    process = subprocess.Popen(["./cloudflared", "tunnel", "--url", f"http://localhost:{PORT}", "--logfile", "cloudflared.log"], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    time.sleep(5)

    # read log for the public URL
    with open("cloudflared.log") as f:
        logs = f.read()
        urls = re.findall(r"https://.*?trycloudflare\.com", logs)
        if urls:
            public_url = urls[-1]
            print("\n✅ Tunnel active!")
            print("Public API URL:", public_url)
            print("\nPOST to: " + public_url + "/generate")
            return public_url
        else:
            print("⚠️ Cloudflared didn't return a URL. Restarting...")
            process.kill()
            time.sleep(3)
            return start_tunnel()

public_url = start_tunnel()

In [ ]:
import requests
res = requests.post("https://strengthening-cast-rights-declined.trycloudflare.com/generate", json={"topic":"creepy hospital facts"})
print(res.json())
jobid = res.json()["jobid"]

In [ ]:
import time, requests
while True:
    r = requests.get(f"https://strengthening-cast-rights-declined.trycloudflare.com/status/{jobid}")
    print(r.json())
    if r.json().get("status") in ("done","error"):
        break
    time.sleep(4)


In [2]:
# Single-cell: Full pipeline (facts -> per-fact TTS -> b-roll + music -> vertical video -> save to Google Drive)
# Paste & run in Google Colab (Notebook A). Then call generate_and_upload("your topic here").

# -------------------- INSTALL / IMPORTS --------------------
!pip install -q yt-dlp moviepy Pillow soundfile transformers accelerate git+https://github.com/rhasspy/python-mpv.git >/dev/null
!apt-get update -qq && apt-get install -y -qq ffmpeg >/dev/null

import os, time, uuid, json, subprocess, threading, math
from pathlib import Path
from moviepy.editor import VideoFileClip, AudioFileClip, CompositeVideoClip, concatenate_videoclips, ColorClip, ImageClip, CompositeAudioClip, afx
from PIL import Image, ImageDraw, ImageFont
import soundfile as sf
import numpy as np
from google.colab import drive

# -------------------- CONFIG --------------------
BASE_DIR = Path("/content/ai_pipeline")
OUTPUT_DIR = BASE_DIR / "output"
AUDIO_DIR = BASE_DIR / "audio"
BROLL_DIR = BASE_DIR / "broll"
FONT_PATH = "/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf"
DRIVE_OUT = Path("/content/drive/MyDrive/ai_fact_videos")
WIDTH, HEIGHT, FPS = 1080, 1920, 25
FACT_COUNT = 5
VOICE_PRESET = "v2/en_speaker_9"  # Bark preset if Bark available
MUSIC_SEARCH = "dark ambient background music creative commons"

for p in (OUTPUT_DIR, AUDIO_DIR, BROLL_DIR, DRIVE_OUT):
    p.mkdir(parents=True, exist_ok=True)

# -------------------- OPTIONAL KEYS (set in Colab env or leave empty) --------------------
GROQ_API_KEY = os.getenv("GROQ_API_KEY", "")
ELEVEN_API_KEY = os.getenv("ELEVEN_API_KEY", "")

# -------------------- MOUNT DRIVE --------------------
try:
    drive.mount('/content/drive', force_remount=False)
except Exception as e:
    print("Drive mount (if needed) error:", e)

# -------------------- TEXT (facts) GENERATOR --------------------
# Try Groq/OpenAI-like endpoint if key present, else fallback to small HF model (flan-t5-small).
def generate_facts(topic, n=FACT_COUNT):
    topic = str(topic)
    if GROQ_API_KEY:
        try:
            import requests
            url = "https://api.groq.com/openai/v1/chat/completions"
            headers = {"Authorization": f"Bearer {GROQ_API_KEY}", "Content-Type": "application/json"}
            prompt = f"Generate {n} dark, creepy, one-line facts about {topic}. Each 6-18 words. Return as newline-separated list."
            data = {"model":"llama-3.1-8b-instant","messages":[{"role":"user","content":prompt}],"max_tokens":300,"temperature":0.8}
            r = requests.post(url, headers=headers, json=data, timeout=30)
            text = r.json().get("choices",[{}])[0].get("message",{}).get("content","")
            lines = [l.strip("-• ").strip() for l in text.splitlines() if l.strip()]
            if len(lines) >= n:
                return lines[:n]
        except Exception as e:
            print("Groq fetch failed, falling back:", e)

    # Fallback: small HF model local (no heavy download).
    try:
        from transformers import pipeline
        gen = pipeline("text2text-generation", model="google/flan-t5-small", device=-1)
        prompt = f"Write {n} short creepy facts about {topic}. Each 1 sentence, under 18 words. Return as newline separated."
        out = gen(prompt, max_length=256, do_sample=True, top_p=0.95, num_return_sequences=1)[0]['generated_text']
        lines = [l.strip() for l in out.replace("\r", "\n").split("\n") if l.strip()]
        if len(lines) >= n:
            return lines[:n]
    except Exception:
        pass

    # Final minimal fallback
    return [f"{topic} fact {i+1}" for i in range(n)]

# -------------------- TTS: try Bark local first, else gTTS fallback --------------------
BARK_AVAILABLE = False
try:
    from bark import generate_audio, preload_models, SAMPLE_RATE
    try:
        preload_models()
    except Exception as e:
        print("Bark preload warning:", e)
    BARK_AVAILABLE = True
    print("Bark available for TTS.")
except Exception as e:
    print("Bark not available, will use gTTS fallback.", e)
    BARK_AVAILABLE = False

def bark_text_to_wav(text, out_wav, voice=VOICE_PRESET):
    try:
        wav = generate_audio(text=text, history_prompt=voice)
        arr = np.array(wav)
        sf.write(out_wav, arr, SAMPLE_RATE)
        return out_wav
    except Exception as e:
        print("Bark synth failed:", e)
        return None

def gtts_text_to_wav(text, out_wav):
    try:
        from gtts import gTTS
        t = gTTS(text=text, lang="en")
        t.save(str(out_wav))
        return str(out_wav)
    except Exception as e:
        print("gTTS failed:", e)
        return None

def ensure_mp3(in_wav, out_mp3):
    try:
        cmd = ["ffmpeg", "-y", "-i", str(in_wav), "-codec:a", "libmp3lame", "-qscale:a", "2", str(out_mp3)]
        subprocess.run(cmd, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        return out_mp3
    except Exception as e:
        print("ffmpeg convert failed:", e)
        return None

def synthesize_fact_audio(fact_text, out_prefix, idx):
    wav = AUDIO_DIR / f"{out_prefix}_fact{idx+1}.wav"
    mp3 = AUDIO_DIR / f"{out_prefix}_fact{idx+1}.mp3"
    ok = None
    if BARK_AVAILABLE:
        ok = bark_text_to_wav(fact_text, str(wav))
    if not ok:
        ok = gtts_text_to_wav(fact_text, str(wav))
    if not ok:
        raise RuntimeError("No TTS available")
    ensure_mp3(str(wav), str(mp3))
    return str(mp3)

# -------------------- Download b-roll (yt-dlp search, CC) --------------------
def download_brolls(topic, count=FACT_COUNT, max_seconds=8):
    clips = []
    queries = [f"{topic} cinematic", "dark cinematic fog", "abandoned building cinematic", f"{topic} b-roll"]
    for q in queries:
        if len(clips) >= count: break
        try:
            out_template = str(BROLL_DIR / "clip_%(id)s.%(ext)s")
            cmd = ["yt-dlp", f"ytsearch1:{q} creative commons", "--no-playlist", "--format", "mp4", "--output", out_template]
            subprocess.run(cmd, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
            # find newest clip file
            candidates = sorted([p for p in BROLL_DIR.iterdir() if p.name.startswith("clip_") and p.suffix in (".mp4", ".m4v")])
            if not candidates: continue
            src = candidates[-1]
            # trim
            dest = BROLL_DIR / (src.stem + "_trim.mp4")
            clip = VideoFileClip(str(src))
            dur = min(max_seconds, clip.duration)
            clip.subclip(0, dur).write_videofile(str(dest), fps=FPS, codec="libx264", audio_codec="aac", threads=2, verbose=False, logger=None)
            clip.close()
            clips.append(str(dest))
        except Exception as e:
            print("b-roll fetch error:", e)
            continue
    # if not enough clips, use color fallback repeated
    while len(clips) < count:
        clips.append(None)
    return clips[:count]

# -------------------- Download ambient music track --------------------
def download_ambient(max_seconds=120):
    try:
        out_template = str(OUTPUT_DIR / "music_%(id)s.%(ext)s")
        cmd = ["yt-dlp", f"ytsearch1:{MUSIC_SEARCH}", "--no-playlist", "--format", "bestaudio", "--output", out_template]
        subprocess.run(cmd, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        candidates = sorted([p for p in OUTPUT_DIR.iterdir() if p.name.startswith("music_")])
        if not candidates:
            return None
        src = candidates[-1]
        trimmed = OUTPUT_DIR / (src.stem + "_trim.mp3")
        cmd2 = ["ffmpeg", "-y", "-i", str(src), "-t", str(max_seconds), "-vn", "-acodec", "libmp3lame", "-qscale:a", "4", str(trimmed)]
        subprocess.run(cmd2, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        return str(trimmed)
    except Exception as e:
        print("music download failed:", e)
        return None

# -------------------- Text -> Image (PIL) helper (auto-wrap, uses textbbox) --------------------
def render_text_image(text, out_image_path, width=WIDTH, padding=80, font_path=FONT_PATH, font_size=56, align="center"):
    try:
        font = ImageFont.truetype(font_path, font_size)
    except Exception:
        font = ImageFont.load_default()
    # temporary draw for measuring
    temp_img = Image.new("RGBA", (width, 2000), (0,0,0,0))
    temp_draw = ImageDraw.Draw(temp_img)
    words = text.split()
    lines = []
    cur = ""
    for w in words:
        test = (cur + " " + w).strip()
        bbox = temp_draw.textbbox((0,0), test, font=font)
        wsize = bbox[2] - bbox[0]
        if wsize > (width - 2*padding):
            if cur:
                lines.append(cur)
            cur = w
        else:
            cur = test
    if cur:
        lines.append(cur)
    # compute height
    bbox = temp_draw.textbbox((0,0), "Ay", font=font)
    line_h = (bbox[3] - bbox[1]) + 10
    img_h = padding + line_h * len(lines) + padding
    img = Image.new("RGBA", (width, max(img_h, 240)), (0,0,0,0))
    draw = ImageDraw.Draw(img)
    y = padding//2
    for line in lines:
        bbox = draw.textbbox((0,0), line, font=font)
        w = bbox[2] - bbox[0]
        if align=="center":
            x = (width - w)//2
        elif align=="left":
            x = padding
        else:
            x = width - padding - w
        draw.text((x, y), line, font=font, fill=(255,255,255,255))
        y += line_h
    img.save(out_image_path)
    return out_image_path

# -------------------- Build vertical video synchronized to per-fact audio --------------------
def build_vertical(facts, voice_mp3s, brolls, music_path, out_path, username="darktruths.hub007"):
    clips = []
    music_clip = AudioFileClip(music_path).volumex(0.12) if music_path else None
    for i, fact in enumerate(facts):
        # choose b-roll or fallback color
        b = brolls[i] if i < len(brolls) else None
        if b:
            try:
                bg = VideoFileClip(b).resize(height=HEIGHT)
                if bg.w < WIDTH: bg = bg.resize(width=WIDTH)
            except Exception:
                bg = ColorClip((WIDTH, HEIGHT), color=(8,8,12)).set_duration(5)
        else:
            bg = ColorClip((WIDTH, HEIGHT), color=(8,8,12)).set_duration(5)
        # audio duration from mp3
        a = AudioFileClip(voice_mp3s[i])
        dur = a.duration + 0.4
        bg = bg.subclip(0, min(dur, bg.duration)) if bg.duration>dur else bg.set_duration(dur)
        # render text image
        txt_img = OUTPUT_DIR / f"text_{uuid.uuid4().hex[:6]}_{i}.png"
        render_text_image(fact, str(txt_img), width=WIDTH, font_size=56)
        txt_clip = ImageClip(str(txt_img)).set_duration(dur).set_position(("center", int(HEIGHT*0.12)))
        # mix voice + music
        if music_clip:
            try:
                music_sub = music_clip.subclip(0, a.duration).volumex(0.12)
                final_audio = CompositeAudioClip([music_sub, a])
            except Exception:
                final_audio = a
        else:
            final_audio = a
        clip = CompositeVideoClip([bg, txt_clip], size=(WIDTH, HEIGHT)).set_duration(dur).set_audio(final_audio)
        clips.append(clip)
    final = concatenate_videoclips(clips, method="compose")
    # watermark
    wm_img = OUTPUT_DIR / "wm.png"
    render_text_image(username, str(wm_img), width=600, font_size=28)
    wm = ImageClip(str(wm_img)).set_duration(final.duration).set_position(("right","bottom")).set_opacity(0.85)
    final = CompositeVideoClip([final, wm])
    final.write_videofile(str(out_path), fps=FPS, codec="libx264", audio_codec="aac", threads=4)
    return str(out_path)

# -------------------- Main function: generate and upload --------------------
def generate_and_upload(topic):
    task_id = uuid.uuid4().hex[:8]
    print("-> Generating facts...")
    facts = generate_facts(topic, n=FACT_COUNT)
    print("Facts:", facts)
    # synthesize audio per fact
    voice_files = []
    for i, fact in enumerate(facts):
        print("TTS fact", i+1)
        mp3 = synthesize_fact_audio(fact, task_id, i)
        voice_files.append(mp3)
    # download b-rolls
    print("Downloading b-roll clips...")
    brolls = download_brolls(topic, count=FACT_COUNT)
    print("b-rolls:", brolls)
    # download music
    print("Downloading ambient music...")
    music = download_ambient(max_seconds=120)
    print("music:", music)
    # build final video
    out_file = OUTPUT_DIR / f"video_{task_id}.mp4"
    print("Building video (this may take a few minutes)...")
    build_vertical(facts, voice_files, brolls, music, out_file, username=os.getenv("USERNAME_WM","darktruths.hub007"))
    # upload to Drive (copy file)
    DRIVE_OUT.mkdir(parents=True, exist_ok=True)
    dest = DRIVE_OUT / out_file.name
    try:
        subprocess.run(["cp", str(out_file), str(dest)], check=True)
        print("Saved to Google Drive:", dest)
    except Exception as e:
        print("Failed to copy to Drive:", e)
    return {"task_id": task_id, "drive_path": str(dest), "local_path": str(out_file), "facts": facts}

# -------------------- USAGE --------------------
print("Ready. Call generate_and_upload('your topic here') to create a video and save it to your Google Drive.")


  error: subprocess-exited-with-error
  
  × git clone --filter=blob:none --quiet https://github.com/rhasspy/python-mpv.git /tmp/pip-req-build-_3e_x0e6 did not run successfully.
  │ exit code: 128
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

× git clone --filter=blob:none --quiet https://github.com/rhasspy/python-mpv.git /tmp/pip-req-build-_3e_x0e6 did not run successfully.
│ exit code: 128
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Drive mount (if needed) error: Mountpoint must not already contain files
Bark not available, will use gTTS fallback. No module named 'bark'
Ready. Call generate_and_upload('your topic here') to c